# 문제 3 :행정동별 서울생활인구 (단기체류 외국인) 
(ds0_open_data)
* 이용API : http://data.seoul.go.kr/dataList/OA-14993/S/1/datasetView.do
---

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#1.-인증키-읽기" data-toc-modified-id="1.-인증키-읽기-0.1">1. 인증키 읽기</a></span></li><li><span><a href="#2.-요청인자-설정-(xml)" data-toc-modified-id="2.-요청인자-설정-(xml)-0.2">2. 요청인자 설정 (xml)</a></span></li><li><span><a href="#3.-url-만들기-/-호출" data-toc-modified-id="3.-url-만들기-/-호출-0.3">3. url 만들기 / 호출</a></span></li><li><span><a href="#4.-XML-파싱" data-toc-modified-id="4.-XML-파싱-0.4">4. XML 파싱</a></span></li></ul></li><li><span><a href="#코드-통합-(XML)" data-toc-modified-id="코드-통합-(XML)-1">코드 통합 (XML)</a></span></li></ul></div>

---
### 1. 인증키 읽기

In [1]:
import os
import src.mylib
keyPath=os.path.join(os.getcwd(), 'src', 'key.properties')
key=src.mylib.getKey(keyPath)
KEY=str(key['02_03'])

### 2. 요청인자 설정 (xml)

In [2]:
TYPE='xml'
SERVICE='SPOP_FORN_TEMP_RESD_DONG'
START_INDEX=str(1)
END_INDEX=str(5)
STDR_DE_ID=str(20200617)

#params=os.path.join(KEY,TYPE,SERVICE,START_INDEX,END_INDEX)
params="/".join([KEY,TYPE,SERVICE,START_INDEX,END_INDEX,STDR_DE_ID])

print (params[30:])

/xml/SPOP_FORN_TEMP_RESD_DONG/1/5/20200617



### 3. url 만들기 / 호출

In [3]:
import urllib
_url='http://openapi.seoul.go.kr:8088/'
url=urllib.parse.urljoin(_url,params)
print(url) # http://openapi.seoul.go.kr:8088/(인증키)/xml/SPOP_FORN_TEMP_RESD_DONG/1/5/20200617

http://openapi.seoul.go.kr:8088/41764d52446c797535394452456345/xml/SPOP_FORN_TEMP_RESD_DONG/1/5/20200617


In [4]:
import requests

data=requests.get(url).text
print (data)

<?xml version="1.0" encoding="UTF-8"?>
<SPOP_FORN_TEMP_RESD_DONG>
<list_total_count>9849</list_total_count>
<RESULT>
<CODE>INFO-000</CODE>
<MESSAGE>정상 처리되었습니다</MESSAGE>
</RESULT>
<row>
<STDR_DE_ID>20200617</STDR_DE_ID>
<TMZON_PD_SE>00</TMZON_PD_SE>
<ADSTRD_CODE_SE>11110515</ADSTRD_CODE_SE>
<TOT_LVPOP_CO>248.866</TOT_LVPOP_CO>
<CHINA_STAYPOP_CO>86.4335</CHINA_STAYPOP_CO>
<ETC_STAYPOP_CO>162.432</ETC_STAYPOP_CO>
</row>
<row>
<STDR_DE_ID>20200617</STDR_DE_ID>
<TMZON_PD_SE>00</TMZON_PD_SE>
<ADSTRD_CODE_SE>11110530</ADSTRD_CODE_SE>
<TOT_LVPOP_CO>71.2628</TOT_LVPOP_CO>
<CHINA_STAYPOP_CO>0</CHINA_STAYPOP_CO>
<ETC_STAYPOP_CO>71.2627</ETC_STAYPOP_CO>
</row>
<row>
<STDR_DE_ID>20200617</STDR_DE_ID>
<TMZON_PD_SE>00</TMZON_PD_SE>
<ADSTRD_CODE_SE>11110540</ADSTRD_CODE_SE>
<TOT_LVPOP_CO>150.1777</TOT_LVPOP_CO>
<CHINA_STAYPOP_CO>86.2246</CHINA_STAYPOP_CO>
<ETC_STAYPOP_CO>63.9532</ETC_STAYPOP_CO>
</row>
<row>
<STDR_DE_ID>20200617</STDR_DE_ID>
<TMZON_PD_SE>00</TMZON_PD_SE>
<ADSTRD_CODE_SE>11110550</ADST

### 4. XML 파싱

In [5]:
import lxml
import lxml.etree

tree=lxml.etree.fromstring(data.encode('utf-8'))
nodes=tree.xpath('//TOT_LVPOP_CO')
for node in nodes:
    print (node.text)

248.866
71.2628
150.1777
136.7137
174.824


---
## 코드 통합 (XML)

In [6]:
%%writefile src/ds_open_foreigners_xml.py
import os
import urllib
import requests
import lxml
import lxml.etree
import mylib #from src import mylib

def doIt():
    keyPath=os.path.join(os.getcwd(), 'src', 'key.properties')
    key=mylib.getKey(keyPath)
    KEY=str(key['02_03'])
    TYPE='xml'
    SERVICE='SPOP_FORN_TEMP_RESD_DONG'
    START_INDEX=str(1)
    END_INDEX=str(10)
    STDR_DE_ID=str(20200617)
    #params="/".join([KEY,TYPE,SERVICE,START_INDEX,END_INDEX,STDR_DE_ID])

    #_url='http://openAPI.seoul.go.kr:8088/'
    _url='http://openAPI.seoul.go.kr:8088' # NOTE: the slash at the end removed
    #url=urllib.parse.urljoin(_url,params)
    url="/".join([_url,KEY,TYPE,SERVICE,START_INDEX,END_INDEX,STDR_DE_ID])
    data=requests.get(url).text
    #tree=lxml.etree.parse(StringIO.StringIO(data.encode('utf-8')))
    tree=lxml.etree.fromstring(data.encode('utf-8'))
    nodes=tree.xpath('//TOT_LVPOP_CO')
    for node in nodes:
        print (node.text)

if __name__ == "__main__":
    doIt()

Writing src/ds_open_foreigners_xml.py


In [7]:
!python src/ds_open_foreigners_xml.py

248.866
71.2628
150.1777
136.7137
174.824
0.4833
0.2163
35.2093
756.5433
87.6117
